In [21]:
import pandas as pd
from sqlalchemy import create_engine
from references_dict import Team_Dictionary
engine = create_engine('mysql://root:@localhost:3306/nfl_db')
file = ("D:\\NFLDB\\game_info.csv")
columns = [
    'temperature',
    'home_favorite',
    'tf_ATS',
    'tf_OU',
    'tu_ATS',
    'tu_OU',
    'tf_rushMatchup_le',
	'tf_rushMatchup_lt',
    'tf_rushMatchup_lg',
    'tf_rushMatchup_m',
    'tf_rushMatchup_re',
    'tf_rushMatchup_rt',
    'tf_rushMatchup_rg',
    'tu_rushMatchup_le',
	'tu_rushMatchup_lt',
    'tu_rushMatchup_lg',
    'tu_rushMatchup_m',
    'tu_rushMatchup_re',
    'tu_rushMatchup_rt',
    'tu_rushMatchup_rg',
    'tf_passMatchup_sl',
    'tf_passMatchup_sm',
    'tf_passMatchup_sr',
    'tf_passMatchup_dl',
    'tf_passMatchup_dm',
    'tf_passMatchup_dr',
    'tu_passMatchup_sl',
    'tu_passMatchup_sm',
    'tu_passMatchup_sr',
    'tu_passMatchup_dl',
    'tu_passMatchup_dm',
    'tu_passMatchup_dr',
    'ATS_Result',
    'OU_Result'
]

def get_home_favorite(row):
	home_team = row['team_home']
	home_abbrev = Team_Dictionary().kaggle_abbrev[home_team]
	if home_abbrev == row['team_favorite_id']:
		return 1
	else:
		return 0
    
def get_spread_result(row):
    score_fav = 0
    score_und = 0
    spread = row['spread_favorite']*-1
    if(row['home_favorite']==1):
        score_fav = row['score_home']
        score_und = row['score_away']
    else:
        score_fav = row['score_away']
        score_und = row['score_home']
    diff = score_fav-score_und
    if( diff > spread ):
        return 1
    elif( diff < spread ):
        return -1
    else:
        return 0
    
def get_OU_result(row):
    OU = float(row['over_under_line'])
    total = row['score_home']+row['score_away']
    if( total > OU ):
        return 1
    elif( total < OU ):
        return -1
    else:
        return 0

    
df = pd.read_csv(file)

# drop playoff weeks
indexNames = df[ df['schedule_week'] == 'WildCard' ].index
df.drop(indexNames,inplace=True)
indexNames = df[ df['schedule_week'] == 'Wildcard' ].index
df.drop(indexNames,inplace=True)
indexNames = df[ df['schedule_week'] == 'Division' ].index
df.drop(indexNames,inplace=True)
indexNames = df[ df['schedule_week'] == 'Conference' ].index
df.drop(indexNames,inplace=True)
indexNames = df[ df['schedule_week'] == 'SuperBowl' ].index
df.drop(indexNames,inplace=True)
indexNames = df[ df['schedule_week'] == 'Superbowl' ].index
df.drop(indexNames,inplace=True)

# drop stats older than 2009
indexNames = df[ df['schedule_season'] < 2009 ].index
df.drop(indexNames,inplace=True)

# # generate metrics for dataset
df['home_favorite'] = df.apply (lambda row: get_home_favorite(row), axis=1)
df['spread_result'] = df.apply(lambda row: get_spread_result(row),axis=1)
df['OU_result'] = df.apply(lambda row: get_OU_result(row),axis=1)

df.schedule_week = df.schedule_week.astype('int')
dfMet = df[df['schedule_week'].between(4,16)]    

gf = pd.DataFrame(columns=columns)
gf['temperature'] = dfMet['weather_temperature']
gf['home_favorite'] = dfMet['home_favorite']

In [19]:
season = 2013
week = 8
team_home = 'Denver Broncos'
team_away = 'Washington Redskins'

last3 = df[df['schedule_season']==season]
last3 = last3[last3['schedule_week']<week]

In [22]:
last3

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,home_favorite,spread_result,OU_result
10809,09/05/2013,2013,1,False,Denver Broncos,49,27,Baltimore Ravens,DEN,-7.5,49.5,Sports Authority Field at Mile High,False,83.0,7.0,30,NaN,1,1,1
10810,09/08/2013,2013,1,False,Buffalo Bills,21,23,New England Patriots,NE,-10.5,51.5,Ralph Wilson Stadium,False,65.0,6.0,55,NaN,0,-1,-1
10811,09/08/2013,2013,1,False,Carolina Panthers,7,12,Seattle Seahawks,SEA,-3.5,45,Bank of America Stadium,False,87.0,7.0,48,NaN,0,1,-1
10812,09/08/2013,2013,1,False,Chicago Bears,24,21,Cincinnati Bengals,CHI,-3.0,42,Soldier Field,False,77.0,13.0,79,NaN,1,0,1
10813,09/08/2013,2013,1,False,Cleveland Browns,10,23,Miami Dolphins,CLE,-2.5,40.5,FirstEnergy Stadium,False,72.0,16.0,80,NaN,1,-1,-1
10814,09/08/2013,2013,1,False,Dallas Cowboys,36,31,New York Giants,DAL,-3.5,50,Cowboys Stadium,False,72.0,0.0,NaN,DOME,1,1,1
10815,09/08/2013,2013,1,False,Detroit Lions,34,24,Minnesota Vikings,DET,-4.5,47.5,Ford Field,False,72.0,0.0,NaN,DOME,1,1,1
10816,09/08/2013,2013,1,False,Indianapolis Colts,21,17,Oakland Raiders,IND,-11.5,45.5,Lucas Oil Stadium,False,72.0,0.0,NaN,DOME,1,-1,-1
10817,09/08/2013,2013,1,False,Jacksonville Jaguars,2,28,Kansas City Chiefs,KC,-4.5,43,EverBank Field,False,87.0,10.0,55,NaN,0,1,-1
10818,09/08/2013,2013,1,False,New Orleans Saints,23,17,Atlanta Falcons,NO,-3.5,56,Louisiana Superdome,False,72.0,0.0,NaN,DOME,1,1,-1
